<a href="https://colab.research.google.com/github/gojiteji/AIorNOT/blob/main/AIORNOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
#import jax.tools.colab_tpu
#jax.tools.colab_tpu.setup_tpu()

In [4]:
!pip install datasets
!pip install flax==0.6.2
!pip install optax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 83.9 MB/s eta 0:00:

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [6]:
from datasets import load_dataset
dataset = load_dataset("competitions/aiornot")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/competitions___parquet/competitions--aiornot-f559c639a48093b2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download('competitions/aiornot', '.extras/sample_submission.csv', repo_type='dataset')
import pandas as pd
df = pd.read_csv(file_path)

In [8]:
import flax
import flax.linen as nn
import optax
import jax.numpy as np

In [9]:
class CNNClassifier(nn.Module):
  def setup(self):
    print("initializing model")
    self.conv1 = nn.Conv(features=32, kernel_size=(3, 3))
    self.conv2 = nn.Conv(features=64, kernel_size=(3, 3))
    self.dense1 = nn.Dense(features=256)
    self.dense2 = nn.Dense(features=1)
  def __call__(self, x):
    x = self.conv1(x)
    x = nn.leaky_relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = self.conv2(x)
    x = nn.leaky_relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    x = self.dense1(x)
    x = nn.leaky_relu(x)
    x = self.dense2(x)
    return x

In [10]:
model=CNNClassifier()

In [11]:
from functools import partial

In [12]:
@partial(jax.pmap,axis_name="devices")
def training_step(state,images,labels):
  def loss_fn(params):
    logits=state.apply_fn({'params': params}, images)
    loss=optax.sigmoid_binary_cross_entropy(logits=logits, labels=labels).mean()
    return loss
  grad_fn = jax.value_and_grad(loss_fn)
  loss, grads = grad_fn(state.params)
  grads = jax.lax.pmean(grads,axis_name="devices")
  state = state.apply_gradients(grads=grads)
  return state, loss

In [15]:
num_epochs = 10
device_count = jax.device_count()
batch_size = 32*device_count

from flax.training.train_state import TrainState
params = model.init(jax.random.PRNGKey(0), np.ones((batch_size//device_count,512,512,3)))['params']
tx = optax.adamw(5e-5)
state = TrainState.create(apply_fn=model.apply, params=params, tx=tx)
from flax import jax_utils
state = jax_utils.replicate(state)
from torch.utils.data import DataLoader
from tqdm import tqdm

initializing model


In [ ]:
dataloader = DataLoader(dataset["train"].with_format("np"), batch_size=batch_size)

loss_list=[]
for epoch in tqdm(range(num_epochs)):
  epoch_loss=[]
  for batch in dataloader:
    images = np.array(batch["image"].reshape([device_count,-1,512,512,3]))
    labels = np.array(batch["label"].reshape([device_count,-1]))
    state, loss = training_step(state,images,labels)
    epoch_loss.append(loss)
  loss_list.append(np.mean(np.array(epoch_loss)))
  print("loss:",np.mean(np.array(epoch_loss)))

 10%|█         | 1/10 [01:49<16:25, 109.45s/it]

loss: 0.8406101


 20%|██        | 2/10 [03:38<14:32, 109.09s/it]

loss: 0.7388352


 30%|███       | 3/10 [05:31<12:57, 111.02s/it]

...

loss: 0.7122706


 70%|███████   | 7/10 [12:42<05:24, 108.24s/it]

loss: 0.7144895


 80%|████████  | 8/10 [14:29<03:35, 107.88s/it]

loss: 0.69766194


In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list);plt.show()

In [ ]:
@jax.jit
def test_step(state,images):
  logits = state.apply_fn({'params': params}, images)
  return logits
print("===test step===")
dataloader = DataLoader(dataset["test"].with_format("np"), batch_size=batch_size//device_count)
logits=None
for batch in tqdm(dataloader):
  images = np.array(batch["image"].reshape([-1,512,512,3]))
  logits = test_step(state,images)

In [ ]:
preds=logits>0.5
df["label"]=preds